In [1]:
import json
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import pickle   
from lifelines import CoxPHFitter # install lifelines-0.30.0 if not already installed
from lifelines.utils import concordance_index


In [2]:
# load test features
test_radiomics = pd.read_csv('D:\\Shruti\\IS_project\\test_radiomics_CA.csv')

# retain patient_id separately; you need the surival data for these patients also (survival time and event columns)
pids = test_radiomics['patient_id'].values
# survival_time = test_radiomics['survival_time'].values
# event = test_radiomics['event'].values
test_radiomics = test_radiomics.drop(columns=['patient_id'])

# remove features that dont have 'original' in their name
keep_cols = [col for col in test_radiomics.columns if ('diagnostics' not in col)] # texture features
# keep_cols = [col for col in test_radiomics.columns if ('diagnostics' not in col) and ('shape' in col)] # shape features only
test_radiomics = test_radiomics[keep_cols]  

# add relevant suffix to each column name, acc. to the ROI (check results.json file used)
test_radiomics.columns = [col + '_CA' for col in test_radiomics.columns]
test_radiomics.head()

,original_shape_Elongation_CA,original_shape_Flatness_CA,original_shape_LeastAxisLength_CA,original_shape_MajorAxisLength_CA,original_shape_Maximum2DDiameterColumn_CA,original_shape_Maximum2DDiameterRow_CA,original_shape_Maximum2DDiameterSlice_CA,original_shape_Maximum3DDiameter_CA,original_shape_MeshVolume_CA,original_shape_MinorAxisLength_CA,...,original_glszm_SmallAreaHighGrayLevelEmphasis_CA,original_glszm_SmallAreaLowGrayLevelEmphasis_CA,original_glszm_ZoneEntropy_CA,original_glszm_ZonePercentage_CA,original_glszm_ZoneVariance_CA,original_ngtdm_Busyness_CA,original_ngtdm_Coarseness_CA,original_ngtdm_Complexity_CA,original_ngtdm_Contrast_CA,original_ngtdm_Strength_CA
0,0.313837,0.291672,26.229081,89.926623,58.777494,30.543422,71.091331,72.980137,341.731863,28.222324,...,38.014409,0.029244,5.682318,0.325770,15.587188,0.458881,0.013565,230.358098,0.076003,2.356547
1,0.456767,0.254310,32.851519,129.179123,94.334191,79.704818,101.589978,115.225281,1754.061699,59.004776,...,260.560771,0.005078,6.618492,0.415656,17.091708,0.327269,0.003310,2212.985515,0.084942,4.361335
2,0.508516,0.357134,39.602673,110.890276,92.476335,57.393386,82.925306,98.877470,1082.391229,56.389476,...,162.972216,0.006063,6.150802,0.478873,4.798097,0.330205,0.005158,1345.317938,0.100797,3.225903
3,0.772971,0.577804,43.648933,75.542864,91.442354,58.548983,74.089791,91.553736,580.868796,58.392450,...,208.465277,0.010738,6.329654,0.432804,9.813117,0.195728,0.009929,2334.566302,0.120726,16.400015
4,0.764122,0.314540,22.555825,71.710443,58.451870,42.348738,80.802630,87.441193,317.978453,54.795513,...,43.349876,0.012709,5.484957,0.364606,8.729113,0.407306,0.020021,170.316479,0.113157,2.494728


In [5]:
results_json = json.load(open('D:\\Shruti\\IS_project\\models\\coronary_arteries_texture\\results.json'))
selected_features = results_json['selected_features']

# make new dataframe X_test from test_radiomics and retain patient_id_NSCLC column
X_test = test_radiomics.copy()

# filter only texture features (remove shape features)
f_names = X_test.columns.tolist()
texture_features = [f for f in f_names if 'shape' not in f]
X_test = X_test[texture_features]

# scale test data using the same scalar used for training
with open('D:\\Shruti\\IS_project\\models\\coronary_arteries_texture\\scalar.pkl', 'rb') as f:
    scaler = pickle.load(f)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns)  


# select only the selected features from test_radiomics
X_test_selected = X_test[selected_features]

# check if the number of selected features matches the number of columns in X_test_selected
print(len(selected_features), len(X_test_selected.columns))

34 34


c:\Users\P70073858\.conda\envs\shruti_env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
# load CoxPH model from pickle file


with open('D:\\Shruti\\IS_project\\models\\coronary_arteries_texture\\coxph_model.pkl', 'rb') as f:
    cph = pickle.load(f)

# predict risk scores for test set
risk_scores = -cph.predict_partial_hazard(X_test_selected)

# calculate C-index --> please ensure you pass the correct survival time and event data for the test set patients here
test_c_index = concordance_index(event_observed=survival_time_column,  # --> edit as needed
                                 predicted_scores=risk_scores,
                                 event_times=event_column) # --> edit as needed
print("C-index on test set: ", test_c_index)

NameError: name 'survival_time_column' is not defined